# test to load local package

## try on dataset c20200927 c20201113 in folder SPOTSAR/test_data


In [8]:
# import external packages
import rasterio as rio

# import main local package
import SPOTSAR_main as sm


In [9]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_file = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
directory_path = "./test_data/CSK_dsc/OFFS/"
# define path to ccp and ccs files
directory_path_ccs = "./test_data/CSK_dsc/CCS/"

# Set the regular expression pattern to match the file names
pattern = r"^c[0-9]+_c[0-9]+_disp_[0-9]+_[0-9]+\.txt$"
# Set the regular expression pattern to match the ccs file names
pattern_ccs = r"^c[0-9]+_c[0-9]+_ccs_[0-9]+_[0-9]+$"



In [10]:
# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_file)
shading = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_extent=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# reorder file using Post_processing.reorder_files
matching_files = sm.Post_processing.reorder_files(directory_path,pattern,0)
matching_files_ccs = sm.Post_processing.reorder_files(directory_path_ccs,pattern_ccs,0)

# test if file ordering has worked
print(matching_files)
print(matching_files_ccs)

['c20200927_c20201113_disp_42_20.txt', 'c20200927_c20201113_disp_74_36.txt', 'c20200927_c20201113_disp_108_52.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_174_84.txt', 'c20200927_c20201113_disp_206_100.txt', 'c20200927_c20201113_disp_240_116.txt', 'c20200927_c20201113_disp_272_132.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_340_164.txt', 'c20200927_c20201113_disp_372_180.txt']
['c20200927_c20201113_ccs_42_20', 'c20200927_c20201113_ccs_74_36', 'c20200927_c20201113_ccs_108_52', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_174_84', 'c20200927_c20201113_ccs_206_100', 'c20200927_c20201113_ccs_240_116', 'c20200927_c20201113_ccs_272_132', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_340_164', 'c20200927_c20201113_ccs_372_180']


In [ ]:

# define dimensions of lat lon files
width = 17478
lines = 14394
# define dimensions of ccs files
width_ccs = 468
lines_ccs = 760

# define starting position of cropped data in lat lon files
r_start = 1324
a_start = 2235
# define step size of the pixel offset tracking to correctely 
# assign lat lon values 
r_step = 24
a_step = 12

# define heading and incidence angle
heading = -167.9496230
mean_inc = 35.6625
# define Multi-kernel-averaging completeness limit
comp_lim = 0.7


# define file names for data, lon and lat
directory_path = "../CSK/dsc1/test_window_size/"
# define path to ccp and ccs files
directory_path_ccp_ccs = "../CSK/dsc1/test_window_size/CCP_CCS/"

# define lon and lat files
lon_file = '../CSK/dsc1/test_window_size/20200910.lon'
lat_file = '../CSK/dsc1/test_window_size/20200910.lat'

datastack = DataStack(directory_path,matching_files,directory_path_ccp_ccs,matching_files_ccs,lat_file,lon_file,heading,mean_inc,lines_ccs,width_ccs)
datastack.get_params_from_file_name()
datastack.get_latlon_from_file(width)
datastack.add_lat_lon_to_data(r_start,a_start)
datastack.crop_stack_ccs(r_step,a_step)
stacked_data = datastack.assign_data_to_stack(r_step,a_step)